In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA_ext.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA_ext.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA__ext.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA_ext.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("MA");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
# week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
day = 3

demandsDict = readall("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM/results/demandsDictFixed$(day).json");
demandsDict = JSON.parse(demandsDict);

In [6]:
# demands = demandsDict["$(length(demandsDict)-1)"]

demands = demandsDict

Dict{UTF8String,Any} with 484 entries:
  "(16,1)"  => 143.31856967684737
  "(11,20)" => 0.0
  "(2,8)"   => 0.0
  "(3,22)"  => 0.0
  "(20,1)"  => 0.0
  "(14,5)"  => 0.0
  "(19,5)"  => 0.0
  "(11,1)"  => 0.0
  "(9,6)"   => 0.0
  "(2,17)"  => 0.0
  "(15,20)" => 8.981141801180812
  "(8,12)"  => 0.5957156868770788
  "(22,20)" => 0.0
  "(8,9)"   => 0.0
  "(14,11)" => 0.0
  "(15,22)" => 0.0
  "(13,16)" => 356.12811525449996
  "(19,2)"  => 0.0
  "(2,18)"  => 0.0
  "(11,17)" => 0.0
  "(11,7)"  => 0.0
  "(3,3)"   => 0.0
  "(15,21)" => 1.0198418364349517
  "(4,1)"   => 302.0451592290054
  "(16,3)"  => 0.0
  ⋮         => ⋮

In [7]:
odPairLabel_;

In [8]:
demandsVec = zeros(462)

for i = 1:length(demandsVec)
    demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
end

In [9]:
demandsVec

462-element Array{Float64,1}:
   0.0   
  11.4166
 200.506 
   0.0   
  18.8793
 121.307 
   0.0   
   0.0   
   0.0   
   0.0   
 214.402 
 160.744 
 219.451 
   ⋮     
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   
   0.0   

In [10]:
for key=keys(odPairRoute)
    if contains(key, "462-")
        println(key)
    end
end

462-329


In [11]:
linkRoute;

In [12]:
deg = 6
c = 0.4
lam = 100.0

coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

# fcoeffs = [1, 0, 0, 0, .15]

7-element Array{Any,1}:
  1.0        
  0.000382575
  0.0334924  
 -0.0260378  
  0.111007   
 -0.0319598  
  0.0227808  

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 74
numRoute = 4614
numOD = 462

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .15*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2265
Number of nonzeros in inequality constraint Jacobian.:     4614
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:     4688
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

74-element Array{Float64,1}:
 1048.48      
 1159.65      
   30.2959    
 2039.83      
   22.5392    
 1897.39      
 1073.87      
 1876.12      
   38.9001    
   37.2498    
    6.36779   
    3.00563   
   -5.0e-9    
    ⋮         
  997.559     
 1319.68      
 1077.89      
 1869.38      
   -9.50002e-8
   -1.00005e-8
   -9.49997e-8
   -9.99991e-9
  104.227     
  284.82      
  504.358     
  922.932     

In [15]:
getObjectiveValue(m)

9550.958723205364

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 74 entries:
  (17,15) => 284.82009274818745
  (3,6)   => 38.90011178887766
  (21,13) => 831.9786105394004
  (22,16) => -1.000045930244723e-8
  (19,12) => -1.9999950445819355e-8
  (22,14) => -9.499971156531339e-8
  (15,17) => 104.22651636236841
  (10,11) => 4.536535210855223
  (13,12) => 0.0
  (3,1)   => 2039.8331500500574
  (16,13) => 1319.6828877647965
  (4,5)   => 1241.6302510816888
  (13,16) => 997.5585828747044
  (17,16) => 922.932150904015
  (20,13) => 640.3343396183525
  (12,8)  => 238.5783005620504
  (2,4)   => 1073.8736149768226
  (16,17) => 504.35844986935484
  (1,2)   => 1048.4799344054427
  (14,21) => 828.2853100783046
  (2,3)   => 22.539154290233203
  (13,19) => -7.99993439766563e-8
  (13,14) => 0.0
  (16,14) => 1869.378315509279
  (8,12)  => 531.8469796036381
  ⋮       => ⋮

In [17]:
# getValue(linkFlow)

In [18]:
# getObjectiveValue(m)

In [19]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

In [20]:
flow_user = GLS_Apr_weekday_PM_ext.x_

74x30 Array{Float64,2}:
 1366.35    791.95    771.324  …  1015.01        1624.03     280.272
 1206.76    992.338   991.158     1000.59        1273.13     570.559
 2114.24   1499.68   1200.6       1171.78        1608.83     947.273
 2273.84   1299.3     980.768     1186.21        1959.73     656.986
   58.465   201.97    271.021       86.5748        40.0646   261.282
  139.064   247.675   201.826  …    23.5801       146.69     143.578
 1193.7    1571.81   1353.56       781.132       1109.21    1362.48 
  953.514  1726.49   1642.59       829.701        651.679   1770.47 
 2124.9    1398.79    938.436     1533.92        1629.54    1007.64 
 2365.09   1244.1     649.406     1485.35        2087.07     599.646
  406.406   514.047   567.011  …   469.666        455.737    655.187
  281.997  1677.2    1591.06       204.042        239.965   1660.44 
  204.522   246.959  1020.38         1.2451e-13   268.26    1131.08 
    ⋮                          ⋱                                    
  817.703 

In [21]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
        free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
        free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
        free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [22]:
# week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Apr_list = [day]

poaDictAprPM = Dict{}()

for i in week_day_Apr_list
    poaDictAprPM[i] = socialObj(flow_user[:, i])/getObjectiveValue(m)
    println(poaDictAprPM[i])
end

2.018485459281115


In [23]:
length(poaDictAprPM)

1

In [24]:
outfile = open("./results/poaDictAprPM$(day).json", "w")

JSON.print(outfile, poaDictAprPM)

close(outfile)